In [1]:
%matplotlib inline
import numpy as np

from transform import combine_tsv_files, load_process_and_store_spreadsheets

In [2]:
def harmonic_mean(t0, t1):
    return t0 * t1 / (t0 + t1)

load_process_and_store_spreadsheets(
    transformation=harmonic_mean,
    all_filename_prefix="output/all__harmonic_mean_",
    train_filename_prefix="output/train__harmonic_mean_",
    test_filename_prefix="output/test__harmonic_mean_",
)

AttributeError: 'int' object has no attribute 'split'

In [ ]:
def difference(t0, t1):
    return t1 - t0

load_process_and_store_spreadsheets(
    transformation=difference,
    all_filename_prefix="output/all__difference_",
    train_filename_prefix="output/train__difference_",
    test_filename_prefix="output/test__difference_",
)

In [ ]:
def relative_difference(t0, t1):
    return (t1-t0)/t0

load_process_and_store_spreadsheets(
    transformation=relative_difference,
    all_filename_prefix="output/all__relative_difference_",
    train_filename_prefix="output/train__relative_difference_",
    test_filename_prefix="output/test__relative_difference_",
)

In [ ]:
def up_or_down(t0, t1):
    return np.sign(t1 - t0)


load_process_and_store_spreadsheets(
    transformation=up_or_down,
    all_filename_prefix="output/all__sign_",
    train_filename_prefix="output/train__sign_",
    test_filename_prefix="output/test__sign_",
)

Data without train/test split, with CNV scores merged in.

In [ ]:
# Harmonic mean genomic variable.
X_hm_af, y_hm_af = combine_tsv_files(
    "output/all__harmonic_mean__Allele Fraction.tsv",
    "output/all__harmonic_mean__CNV Score.tsv",
)

X_hm_af.merge(y_hm_af, left_index=True, right_index=True).to_excel('output/all__harmonic_mean__af_cnv.xlsx')

In [ ]:
# Harmonic mean genomic variable.
X_hm_molecules, y_hm_molecules = combine_tsv_files(
    "output/all__harmonic_mean__No. Mutant Molecules per mL.tsv",
    "output/all__harmonic_mean__CNV Score.tsv",
)

X_hm_molecules.merge(y_hm_molecules, left_index=True, right_index=True).to_excel('output/all__harmonic_mean__molecules_cnv.xlsx')